In [1]:
# testing again after chaning MCTCSelfOutput to this differnt way flashlight seems to do things

# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model = model.eval()
our_self = model.mctc.encoder.layers[0].attention.self
first_layer = model.mctc.encoder.layers[0]


In [2]:
import arrayfire as af

model = model.float()
layer_x = af.array.read_array("OUTPUT_TRF.arr", key="layer_x").to_ndarray()

def valueprint(k, name=None):
    print(name, k.shape, k.sum(), k.std())
our_output = torch.tensor(layer_x).unsqueeze(0).transpose(1,2)
print(our_output.shape, "INPUT")

context_layer, attention_scores, attention_probs, value_layer, save_dict = our_self(our_output, output_attentions=True)

torch.Size([1, 321, 1536]) INPUT
attention_scores torch.Size([1, 4, 321, 321]) tensor(87683.4062, grad_fn=<SumBackward0>) tensor(0.9123, grad_fn=<StdBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>) tensor(0.1639, grad_fn=<StdBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1017.8952, grad_fn=<SumBackward0>) tensor(0.0613, grad_fn=<StdBackward0>)


In [3]:
'''Softmax of above final pscores
'''
our_self = model.mctc.encoder.layers[0].attention.self
context_layer, attention_scores, attention_probs, value_layer, save_dict = our_self(our_output, output_attentions=True)
mapper = {
    "MHAttention_scores": save_dict["pre_posemb_attention_scores"],
    "MHAttention_matmulPos": save_dict["relative_position_scores_1"],
    "MHAttention_pscores_transposed": save_dict["relative_position_scores_2"],
    "MHAttention_pscores_scores": save_dict["attention_scores"],
    "MHAttention_result": save_dict["post_posemb_attention_scores"],
    "MHAttention_result_attn": save_dict["attention_probs"],
    # "MHAttention_value": save_dict["value_layer"],
    "MHAttention_v": save_dict["value_layer"],
    "MHAttention_result_result": save_dict["context_layer"],
    "MHAttention_result_result_2": context_layer
}
o2o = {}
for key, myval in mapper.items():
    fl_val = af.array.read_array("OUTPUT_TRF.arr", key=key).to_ndarray()
    o2o[key] = {
        "fl": fl_val,
        "my": myval,
    }

    valueprint(fl_val, key)
    
    # valueprint(myval.detach().numpy(), key)
    if key == "MHAttention_result_result_2":
        myval = myval.squeeze(0).detach().numpy()
    else:
        myval = myval.squeeze(0).permute(1,2,0).detach().numpy()
    valueprint(myval, key)

    diff = fl_val - myval
    valueprint(diff, "DIFF")
    print()


attention_scores torch.Size([1, 4, 321, 321]) tensor(87683.4062, grad_fn=<SumBackward0>) tensor(0.9123, grad_fn=<StdBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, grad_fn=<SumBackward0>) tensor(0.1639, grad_fn=<StdBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1017.8952, grad_fn=<SumBackward0>) tensor(0.0613, grad_fn=<StdBackward0>)
MHAttention_scores (321, 321, 4) 30256.422 0.9070506
MHAttention_scores (321, 321, 4) 30256.418 0.9070506
DIFF (321, 321, 4) 0.0004733706 2.862019e-07

MHAttention_matmulPos (1839, 321, 4) -85057.48 0.13692527
MHAttention_matmulPos (1839, 321, 4) -85057.41 0.13692527
DIFF (1839, 321, 4) -5.647499e-06 4.3446907e-08

MHAttention_pscores_transposed (321, 321, 4) 57426.992 0.19243532
MHAttention_pscores_transposed (321, 321, 4) 57426.99 0.19243534
DIFF (321, 321, 4) 1.8553577e-05 7.244098e-08

MHAttention_pscores_scores (321, 321, 4) 87683.41 0.91226983
MHAttention_pscores_scores (321, 321, 4) 87683.41 0.9122699
DIFF (321

In [4]:
selfAttention_wf_ = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_wf_").to_ndarray()
selfAttention_result = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_result").to_ndarray()
GOLD_OUTPUT = af.array.read_array("OUTPUT_TRF.arr", key="selfAttention_result_2").to_ndarray()
my_wf = model.mctc.encoder.layers[0].attention.output.dense.weight
my_result = context_layer

valueprint(selfAttention_wf_)
valueprint(my_wf)
print()
valueprint(selfAttention_result)
valueprint(my_result)

None (1536, 1536) -44.906384 0.017451134
None torch.Size([1536, 1536]) tensor(-44.9064, grad_fn=<SumBackward0>) tensor(0.0175, grad_fn=<StdBackward0>)

None (321, 1536) -1017.8952 0.061281413
None torch.Size([1, 321, 1536]) tensor(-1017.8952, grad_fn=<SumBackward0>) tensor(0.0613, grad_fn=<StdBackward0>)


In [5]:
ll = torch.nn.Linear(1536, 1536, bias=False)
ll.weight = torch.nn.Parameter(my_wf)
hidden_states_gold = torch.matmul(torch.tensor(my_wf, dtype=torch.float64), torch.tensor(selfAttention_result, dtype=torch.float64).transpose(-1, -2))
hidden_states_mine = torch.matmul(torch.tensor(my_wf, dtype=torch.float64), torch.tensor(my_result, dtype=torch.float64).transpose(-1, -2))
hidden_states_einsum = torch.einsum('hh, bhe -> bhe', my_wf.float(), my_result.transpose(-1, -2))
hideen_states_linear = ll(my_result)
valueprint(hidden_states_gold)
valueprint(hidden_states_mine)
valueprint(hidden_states_einsum)
valueprint(hideen_states_linear)

None torch.Size([1536, 321]) tensor(25.7065, dtype=torch.float64) tensor(0.1095, dtype=torch.float64)
None torch.Size([1, 1536, 321]) tensor(25.7065, dtype=torch.float64) tensor(0.1095, dtype=torch.float64)
None torch.Size([1, 1536, 321]) tensor(18.9476, grad_fn=<SumBackward0>) tensor(0.0010, grad_fn=<StdBackward0>)
None torch.Size([1, 321, 1536]) tensor(25.7065, grad_fn=<SumBackward0>) tensor(0.1095, grad_fn=<StdBackward0>)


/tmp/ipykernel_17571/2236328910.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hidden_states_gold = torch.matmul(torch.tensor(my_wf, dtype=torch.float64), torch.tensor(selfAttention_result, dtype=torch.float64).transpose(-1, -2))
/tmp/ipykernel_17571/2236328910.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hidden_states_mine = torch.matmul(torch.tensor(my_wf, dtype=torch.float64), torch.tensor(my_result, dtype=torch.float64).transpose(-1, -2))


In [6]:
# testing again after chaning MCTCSelfOutput to this differnt way flashlight seems to do things

# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model = model.eval()
our_attn = model.mctc.encoder.layers[0].attention
first_layer = model.mctc.encoder.layers[0]

# lets take a deeper look.
import arrayfire as af

model = model.double()
layer_x = af.array.read_array("OUTPUT_TRF.arr", key="layer_x").to_ndarray()

def valueprint(k, name=None):
    print(name, k.shape, k.sum(), k.std())
our_output = torch.tensor(layer_x, dtype=torch.float64).unsqueeze(0).transpose(1,2)
context_layer, attention_scores, attention_probs, value_layer, save_dict = our_attn(our_output, output_attentions=True)

attention_scores torch.Size([1, 4, 321, 321]) tensor(87683.4148, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.9123, dtype=torch.float64, grad_fn=<StdBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.1639, dtype=torch.float64, grad_fn=<StdBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1017.8953, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.0613, dtype=torch.float64, grad_fn=<StdBackward0>)
SelfOUtput prenorm torch.Size([1, 321, 1536]) tensor(25.7065, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.1095, dtype=torch.float64, grad_fn=<StdBackward0>)
SelfOUtput input_tensor torch.Size([1, 321, 1536]) tensor(330.5733, dtype=torch.float64)
SelfOUtput postnorm torch.Size([1, 321, 1536]) tensor(-614.8109, dtype=torch.float64, grad_fn=<SumBackward0>)


In [7]:
GOLD_OUTPUT = af.array.read_array("OUTPUT_TRF.arr", key="layer_selfAttnResult").to_ndarray()
valueprint(GOLD_OUTPUT)

None (1536, 321) 25.706522 0.1094709


In [8]:
'''
af::saveArray("layer_selfAttnResult", input_arr, savePathChar, true);

auto h = (*norm1_)((f * selfAttnResult.as(x.type())) + x);

af::saveArray("layer_h", input_arr, savePathChar, true); 
'''
layer_x = af.array.read_array("OUTPUT_TRF.arr", key="layer_x").to_ndarray()
GOLD_OUTPUT_selfAttn = af.array.read_array("OUTPUT_TRF.arr", key="layer_selfAttnResult").to_ndarray()
GOLD_OUTPUT_postnorm = af.array.read_array("OUTPUT_TRF.arr", key="layer_h").to_ndarray()
valueprint(layer_x)
valueprint(GOLD_OUTPUT_selfAttn)
valueprint(GOLD_OUTPUT_postnorm)

None (1536, 321) 330.57333 0.15868725
None (1536, 321) 25.706522 0.1094709
None (1536, 321) -1035.2311 1.179761


In [9]:
my_x = torch.Tensor(layer_x).transpose(0,1)
my_selfAttnResult = torch.Tensor(GOLD_OUTPUT_selfAttn).transpose(0,1)

output_norm = model.mctc.encoder.layers[0].attention.output.LayerNorm

valueprint(my_x)
valueprint(my_selfAttnResult)

my_sum = my_selfAttnResult.transpose(-1, -2) + my_x.transpose(-1, -2)
my_sum = my_sum.transpose(-1, -2)
my_postnorm = output_norm(my_sum)
valueprint(my_postnorm)


None torch.Size([321, 1536]) tensor(330.5734) tensor(0.1587)
None torch.Size([321, 1536]) tensor(25.7066) tensor(0.1095)
None torch.Size([321, 1536]) tensor(-614.8109, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.2113, dtype=torch.float64, grad_fn=<StdBackward0>)


In [10]:
layer_norm1_w = af.array.read_array("OUTPUT_TRF.arr", key="layer_norm1_w").to_ndarray()
layer_norm1_b = af.array.read_array("OUTPUT_TRF.arr", key="layer_norm1_b").to_ndarray()

valueprint(layer_norm1_w)
valueprint(layer_norm1_b)

valueprint(output_norm.singleton_weight)
valueprint(output_norm.singleton_bias)

None (1,) 1.180032 0.0
None (1,) -0.0020996246 0.0
None torch.Size([1]) tensor(1.1800, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(nan, dtype=torch.float64, grad_fn=<StdBackward0>)
None torch.Size([1]) tensor(-0.0021, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(nan, dtype=torch.float64, grad_fn=<StdBackward0>)


In [15]:
layernorm = torch.nn.LayerNorm((1536), eps=1e-05, elementwise_affine=True, device=None, dtype=None)
layernorm.weight = torch.nn.Parameter(output_norm.singleton_weight.float().tile((1536,)))
layernorm.bias = torch.nn.Parameter(output_norm.singleton_bias.float().tile((1536,)))

In [16]:
my_postnorm = layernorm(my_sum)
valueprint(my_postnorm)

None torch.Size([321, 1536]) tensor(-1035.2329, grad_fn=<SumBackward0>) tensor(1.1798, grad_fn=<StdBackward0>)


In [1]:
# adjusted the weights accordingly
# testing again after chaning MCTCSelfOutput to this differnt way flashlight seems to do things

# testing after fixing the model & weights
import torch
from transformers import MCTCForCTC, MCTCProcessor, MCTCConfig
config = MCTCConfig()
model = MCTCForCTC(config)
model.load_state_dict(torch.load("./ported_pytorch_model.bin"))
model = model.eval()
our_attn = model.mctc.encoder.layers[0].attention
first_layer = model.mctc.encoder.layers[0]

# lets take a deeper look.
import arrayfire as af

model = model.double()
layer_x = af.array.read_array("OUTPUT_TRF.arr", key="layer_x").to_ndarray()

def valueprint(k, name=None):
    print(name, k.shape, k.sum(), k.std())
our_output = torch.tensor(layer_x, dtype=torch.float64).unsqueeze(0).transpose(1,2)
context_layer, attention_scores, attention_probs, value_layer, save_dict = our_attn(our_output, output_attentions=True)

attention_scores torch.Size([1, 4, 321, 321]) tensor(87683.4148, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.9123, dtype=torch.float64, grad_fn=<StdBackward0>)
value_layer torch.Size([1, 4, 321, 384]) tensor(-995.1271, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.1639, dtype=torch.float64, grad_fn=<StdBackward0>)
context_layer torch.Size([1, 4, 321, 384]) tensor(-1017.8953, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.0613, dtype=torch.float64, grad_fn=<StdBackward0>)
SelfOUtput prenorm torch.Size([1, 321, 1536]) tensor(25.7065, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.1095, dtype=torch.float64, grad_fn=<StdBackward0>)
SelfOUtput input_tensor torch.Size([1, 321, 1536]) tensor(330.5733, dtype=torch.float64)
SelfOUtput postnorm torch.Size([1, 321, 1536]) tensor(-1035.2325, dtype=torch.float64, grad_fn=<SumBackward0>)


In [2]:
valueprint(context_layer)

None torch.Size([1, 321, 1536]) tensor(-1035.2325, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(1.1798, dtype=torch.float64, grad_fn=<StdBackward0>)


In [5]:
valueprint(output[0])

None torch.Size([1, 321, 1536]) tensor(14736.9198, dtype=torch.float64, grad_fn=<SumBackward0>) tensor(0.4444, dtype=torch.float64, grad_fn=<StdBackward0>)


In [6]:
layer_h_2 = af.array.read_array("OUTPUT_TRF.arr", key="layer_h_2").to_ndarray()
valueprint(layer_h_2)

None (1536, 321) 14736.917 0.4444032


In [7]:
full_output = model(our_output)


RuntimeError: Given groups=1, weight of size [3072, 80, 7], expected input[1, 1536, 327] to have 80 channels, but got 1536 channels instead